In [ ]:
!pip install unzip
!pip install pandas
!pip install tqdm
!pip install python-dotenv
!pip install neo4j
!pip install tenacity
!pip install langchain
!pip install langchain-google-genai
!pip install langchain-ollama
!pip install langchain-community
!pip install langchain-experimental

In [ ]:
# !pip install -q kaggle
# !pip install google.colab
# from google.colab import files

# files.upload()

# !mkdir ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets list

# !kaggle datasets download -d jrobischon/wikipedia-movie-plots
# !unzip wikipedia-movie-plots.zip

In [ ]:
# !kaggle datasets download -d jrobischon/wikipedia-movie-plots
!unzip wikipedia-movie-plots.zip

In [ ]:
# Type hints
from typing import Any, Dict, List, Tuple

# Standard library
import ast
import logging
import re
import warnings

# Third-party packages - Data manipulation
import pandas as pd
from tqdm import tqdm
# from .autonotebook import tqdm as notebook_tqdm

# Third-party packages - Environment & Database
from dotenv import load_dotenv
from neo4j import GraphDatabase

# Third-party packages - Error handling & Retry logic
from tenacity import retry, stop_after_attempt, wait_exponential

# Langchain - Core
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document

# Langchain - Models & Connectors
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAI
from langchain_ollama.llms import OllamaLLM

# Langchain - Graph & Experimental
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer

# Suppress warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

In [ ]:
movies = pd.read_csv('wiki_movie_plots_deduped.csv') # adjust the path if you manually downloaded the dataset
movies.head()

In [ ]:
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """Clean and preprocess DataFrame.

    Args:
        data: Input DataFrame

    Returns:
        Cleaned DataFrame
    """
    df.drop(["Wiki Page"], axis=1, inplace=True)

    # Drop duplicates
    df = df.drop_duplicates(subset='Title', keep='first')

    # Get object columns
    col_obj = df.select_dtypes(include=["object"]).columns

    # Clean string columns
    for col in col_obj:
        # Strip whitespace
        df[col] = df[col].str.strip()

        # Replace unknown/empty values
        df[col] = df[col].apply(
            lambda x: None if pd.isna(x) or x.lower() in ["", "unknown"]
            else x.capitalize()
        )

    # Drop rows with any null values
    df = df.dropna(how="any", axis=0)

    return df

movies = clean_data(movies).head(1000)
movies.head()

In [ ]:
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()
        print("Connection closed")

    def reset_database(self):
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")
            print("Database resetted successfully!")

    def execute_query(self, query, parameters=None):
        with self.driver.session() as session:
            result = session.run(query, parameters or {})
            return [record for record in result]

In [ ]:
uri = "bolt://localhost:7687"
user = "neo4j"
password = "ilovemovies"
conn = Neo4jConnection(uri, user, password)
conn.reset_database()

In [ ]:
def parse_number(value: Any, target_type: type) -> Optional[float]:
    """Parse string to number with proper error handling."""
    if pd.isna(value):
        return None
    try:
        cleaned = str(value).strip().replace(',', '')
        return target_type(cleaned)
    except (ValueError, TypeError):
        return None

def clean_text(text: str) -> str:
    """Clean and normalize text fields."""
    if pd.isna(text):
        return ""
    return str(text).strip().title()

In [ ]:
def load_movies_to_neo4j(movies_df: pd.DataFrame, connection: GraphDatabase) -> None:
    """Load movie data into Neo4j with progress tracking and error handling."""

    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)

    # Query templates
    MOVIE_QUERY = """
        MERGE (movie:Movie {title: $title})
        SET movie.year = $year,
            movie.origin = $origin,
            movie.genre = $genre,
            movie.plot = $plot
    """

    DIRECTOR_QUERY = """
        MATCH (movie:Movie {title: $title})
        MERGE (director:Director {name: $name})
        MERGE (director)-[:DIRECTED]->(movie)
    """

    ACTOR_QUERY = """
        MATCH (movie:Movie {title: $title})
        MERGE (actor:Actor {name: $name})
        MERGE (actor)-[:ACTED_IN]->(movie)
    """

    # Process each movie
    for _, row in tqdm(movies_df.iterrows(), total=len(movies_df), desc="Loading movies"):
        try:
            # Prepare movie parameters
            movie_params = {
                "title": clean_text(row["Title"]),
                "year": parse_number(row["Release Year"], int),
                "origin": clean_text(row["Origin/Ethnicity"]),
                "genre": clean_text(row["Genre"]),
                "plot": str(row["Plot"]).strip()
            }

            # Create movie node
            connection.execute_query(MOVIE_QUERY, parameters=movie_params)

            # Process directors
            for director in str(row["Director"]).split(" and "):
                director_params = {
                    "name": clean_text(director),
                    "title": movie_params["title"]
                }
                connection.execute_query(DIRECTOR_QUERY, parameters=director_params)

            # Process cast
            if pd.notna(row["Cast"]):
                for actor in row["Cast"].split(","):
                    actor_params = {
                        "name": clean_text(actor),
                        "title": movie_params["title"]
                    }
                    connection.execute_query(ACTOR_QUERY, parameters=actor_params)

        except Exception as e:
            logger.error(f"Error loading {row['Title']}: {str(e)}")
            continue

    logger.info("Finished loading movies to Neo4j")

In [ ]:
load_movies_to_neo4j(movies, conn)

In [ ]:
query = """
MATCH (m:Movie)-[:ACTED_IN]-(a:Actor)
RETURN m.title, a.name
LIMIT 10;
"""
results = conn.execute_query(query)
for record in results:
    print(record)

In [ ]:
# llm = GoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=api_key) # if you are using Google API
llm = OllamaLLM(model="qwen2.5-coder:latest")

In [ ]:
node_structure = "\n".join([
    f"{col}: {', '.join(map(str, movies[col].unique()[:3]))}..."
    for col in movies.columns
])
print(node_structure)

In [ ]:
# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def validate_node_definition(node_def: Dict) -> bool:
    """Validate node definition structure"""
    if not isinstance(node_def, dict):
        return False
    return all(
        isinstance(v, dict) and all(isinstance(k, str) for k in v.keys())
        for v in node_def.values()
    )

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def get_node_definitions(chain, structure: str, example: Dict) -> Dict[str, Dict[str, str]]:
    """Get node definitions with retry logic"""
    try:
        # Get response from LLM
        response = chain.invoke({"structure": structure, "example": example})

        # Parse response
        node_defs = ast.literal_eval(response)

        # Validate structure
        if not validate_node_definition(node_defs):
            raise ValueError("Invalid node definition structure")

        return node_defs

    except (ValueError, SyntaxError) as e:
        logger.error(f"Error parsing node definitions: {e}")
        raise

# Updated node definition template
node_example = {
    "NodeLabel1": {"property1": "row['property1']", "property2": "row['property2']"},
    "NodeLabel2": {"property1": "row['property1']", "property2": "row['property2']"},
    "NodeLabel3": {"property1": "row['property1']", "property2": "row['property2']"},
}

define_nodes_prompt = PromptTemplate(
    input_variables=["example", "structure"],
    template=("""
        Analyze the dataset structure below and extract the entity labels for nodes and their properties.\n
        The node properties should be based on the dataset columns and their values.\n
        Return the result as a dictionary where the keys are the node labels and the values are the node properties.\n\n
        Example: {example}\n\n

        Dataset Structure:\n{structure}\n\n

        Make sure to include all the possible node labels and their properties.\n
        If a property can be its own node, include it as a separate node label.\n
        Please do not report triple backticks to identify a code block, just return the list of tuples.\n
        Return only the dictionary containing node labels and properties, and don't include any other text or quotation.

        """
    ),
)

# Execute with error handling
try:
    node_chain = define_nodes_prompt | llm

    node_definitions = get_node_definitions(node_chain, structure=node_structure, example=node_example)
    logger.info(f"Node Definitions: {node_definitions}")
except Exception as e:
    logger.error(f"Failed to get node definitions: {e}")
    raise

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
def validate_node_definition(node_def: Dict) -> bool:
    """Validate node definition structure"""
    if not isinstance(node_def, dict):
        return False
    return all(
        isinstance(v, dict) and all(isinstance(k, str) for k in v.keys())
        for v in node_def.values()
    )

In [ ]:
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def get_node_definitions(chain, structure: str, example: Dict) -> Dict[str, Dict[str, str]]:
    """Get node definitions with retry logic"""
    try:
        # Get response from LLM
        response = chain.invoke({"structure": structure, "example": example})

        # Parse response
        node_defs = ast.literal_eval(response)

        # Validate structure
        if not validate_node_definition(node_defs):
            raise ValueError("Invalid node definition structure")

        return node_defs
    except (ValueError, SyntaxError) as e:
        logger.error(f"Error parsing node definitions: {e}")
        raise

In [ ]:
define_nodes_prompt = PromptTemplate(
    input_variables=["example", "structure"],
    template=("""
        Analyze the dataset structure below and extract the entity labels for nodes and their properties.\n
        The node properties should be based on the dataset columns and their values.\n
        Return the result as a dictionary where the keys are the node labels and the values are the node properties.\n\n
        Example: {example}\n\n
        Dataset Structure:\n{structure}\n\n
        Make sure to include all the possible node labels and their properties.\n
        If a property can be its own node, include it as a separate node label.\n
        Please do not report triple backticks to identify a code block, just return the list of tuples.\n
        Return only the dictionary containing node labels and properties, and don't include any other text or quotation.
    """),
)

In [ ]:
node_example = {
    "NodeLabel1": {"property1": "row['property1']", "property2": "row['property2']"},
    "NodeLabel2": {"property1": "row['property1']", "property2": "row['property2']"},
    "NodeLabel3": {"property1": "row['property1']", "property2": "row['property2']"},
}

In [ ]:
try:
    node_chain = define_nodes_prompt | llm
    node_definitions = get_node_definitions(node_chain, structure=node_structure, example=node_example)
    logger.info(f"Node Definitions: {node_definitions}")
except Exception as e:
    logger.error(f"Failed to get node definitions: {e}")
    raise

In [ ]:
class RelationshipIdentifier:
    """Identifies relationships between nodes in a graph database."""

    RELATIONSHIP_EXAMPLE = [
        ("NodeLabel1", "RelationshipLabel", "NodeLabel2"),
        ("NodeLabel1", "RelationshipLabel", "NodeLabel3"),
        ("NodeLabel2", "RelationshipLabel", "NodeLabel3"),
    ]


    PROMPT_TEMPLATE = PromptTemplate(
    input_variables=["structure", "node_definitions", "example"],
    template="""
        Consider the following Dataset Structure:\n{structure}\n\n

        Consider the following Node Definitions:\n{node_definitions}\n\n

        Based on the dataset structure and node definitions, identify relationships (edges) between nodes.\n
        Return the relationships as a list of triples where each triple contains the start node label, relationship label, and end node label, and each triple is a tuple.\n
        Please return only the list of tuples. Please do not report triple backticks to identify a code block, just return the list of tuples.\n\n

        Example:\n{example}
        """
)

    def __init__(self, llm: Any, logger: logging.Logger = None):
        self.llm = llm
        self.logger = logger or logging.getLogger(__name__)
        self.chain = self.PROMPT_TEMPLATE | self.llm

    def validate_relationships(self, relationships: List[Tuple]) -> bool:
        """Validate relationship structure."""
        return all(
            isinstance(rel, tuple) and
            len(rel) == 3 and
            all(isinstance(x, str) for x in rel)
            for rel in relationships
        )

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
    def identify_relationships(self, structure: str, node_definitions: Dict) -> List[Tuple]:
        """Identify relationships with retry logic."""
        try:
            response = self.chain.invoke({
                "structure": structure,
                "node_definitions": str(node_definitions),
                "example": str(self.RELATIONSHIP_EXAMPLE)
            })

            relationships = ast.literal_eval(response)

            if not self.validate_relationships(relationships):
                raise ValueError("Invalid relationship structure")

            self.logger.info(f"Identified {len(relationships)} relationships")
            return relationships

        except Exception as e:
            self.logger.error(f"Error identifying relationships: {e}")
            raise

    def get_relationship_types(self) -> List[str]:
        """Extract unique relationship types."""
        return list(set(rel[1] for rel in self.identify_relationships()))

# Usage
identifier = RelationshipIdentifier(llm=llm)
relationships = identifier.identify_relationships(node_structure, node_definitions)
print("Relationships:", relationships)

In [ ]:
RELATIONSHIP_EXAMPLE = [
    ("NodeLabel1", "RelationshipLabel", "NodeLabel2"),
    ("NodeLabel1", "RelationshipLabel", "NodeLabel3"),
    ("NodeLabel2", "RelationshipLabel", "NodeLabel3"),
]

In [ ]:
PROMPT_TEMPLATE = PromptTemplate(
    input_variables=["structure", "node_definitions", "example"],
    template="""
        Consider the following Dataset Structure:\n{structure}\n\n

        Consider the following Node Definitions:\n{node_definitions}\n\n

        Based on the dataset structure and node definitions, identify relationships (edges) between nodes.\n
        Return the relationships as a list of triples where each triple contains the start node label, relationship label, and end node label, and each triple is a tuple.\n
        Please return only the list of tuples. Please do not report triple backticks to identify a code block, just return the list of tuples.\n\n

        Example:\n{example}
        """
)

In [ ]:
def __init__(self, llm: Any, logger: logging.Logger = None):
    self.llm = llm
    self.logger = logger or logging.getLogger(__name__)
    self.chain = self.PROMPT_TEMPLATE | self.llm

In [ ]:
def validate_relationships(self, relationships: List[Tuple]) -> bool:
    """Validate relationship structure."""
    return all(
        isinstance(rel, tuple) and
        len(rel) == 3 and
        all(isinstance(x, str) for x in rel)
        for rel in relationships
    )

In [ ]:
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def identify_relationships(self, structure: str, node_definitions: Dict) -> List[Tuple]:
    """Identify relationships with retry logic."""
    try:
        response = self.chain.invoke({
            "structure": structure,
            "node_definitions": str(node_definitions),
            "example": str(self.RELATIONSHIP_EXAMPLE)
        })

        relationships = ast.literal_eval(response)

        if not self.validate_relationships(relationships):
            raise ValueError("Invalid relationship structure")

        self.logger.info(f"Identified {len(relationships)} relationships")
        return relationships
    except Exception as e:
        self.logger.error(f"Error identifying relationships: {e}")
        raise

In [ ]:
def get_relationship_types(self) -> List[str]:
    """Extract unique relationship types."""
    return list(set(rel[1] for rel in self.identify_relationships()))

In [ ]:
identifier = RelationshipIdentifier(llm=llm)
relationships = identifier.identify_relationships(node_structure, node_definitions)
print("Relationships:", relationships)

In [ ]:
class CypherQueryBuilder:
    """Builds Cypher queries for Neo4j graph database."""

    INPUT_EXAMPLE = """
    NodeLabel1: value1, value2
    NodeLabel2: value1, value2
    """

    EXAMPLE_CYPHER = example_cypher = """
    CREATE (n1:NodeLabel1 {property1: "row['property1']", property2: "row['property2']"})
    CREATE (n2:NodeLabel2 {property1: "row['property1']", property2: "row['property2']"})
    CREATE (n1)-[:RelationshipLabel]->(n2);
    """

    PROMPT_TEMPLATE = PromptTemplate(
    input_variables=["structure", "node_definitions", "relationships", "example"],
    template="""
        Consider the following Node Definitions:\n{node_definitions}\n\n
        Consider the following Relationships:\n{relationships}\n\n
        Generate Cypher queries to create nodes and relationships using the node definitions and relationships below. Remember to replace the placeholder values with actual data from the dataset.\n
        Include all the properties in the Node Definitions for each node as defined and create relationships.\n
        Return a single string with each query separated by a semicolon.\n
        Don't include any other text or quotation marks in the response.\n
        Please return only the string containing Cypher queries. Please do not report triple backticks to identify a code block.\n\n

        Example Input:\n{input}\n\n

        Example Output Cypher query:\n{cypher}
    """
)

    def __init__(self, llm: Any, logger: logging.Logger = None):
        self.llm = llm
        self.logger = logger or logging.getLogger(__name__)
        # self.chain = LLMChain(llm=llm, prompt=self.PROMPT_TEMPLATE)
        self.chain = self.PROMPT_TEMPLATE | self.llm

    def validate_cypher_query(self, query: str) -> bool:
        """Validate Cypher query syntax using LLM and regex patterns."""

        VALIDATION_PROMPT = PromptTemplate(
            input_variables=["query"],
            template="""
            Validate this Cypher query and return TRUE or FALSE:

            Query: {query}

            Rules to check:
            1. Valid CREATE statements
            2. Proper property formatting
            3. Valid relationship syntax
            4. No missing parentheses
            5. Valid property names
            6. Valid relationship types

            Return only TRUE if query is valid, FALSE if invalid.
            """
        )

        try:
            # Basic pattern validation
            basic_valid = all(re.search(pattern, query) for pattern in [
                r'CREATE \(',
                r'\{.*?\}',
                r'\)-\[:.*?\]->'
            ])

            if not basic_valid:
                return False

            # LLM validation
            validation_chain = VALIDATION_PROMPT | self.llm
            result = validation_chain.invoke({"query": query})

            # Parse result
            is_valid = "TRUE" in result.upper()

            if not is_valid:
                self.logger.warning(f"LLM validation failed for query: {query}")

            return is_valid

        except Exception as e:
            self.logger.error(f"Validation error: {e}")
            return False

    def sanitize_query(self, query: str) -> str:
        """Sanitize and format Cypher query."""
        return (query
                .strip()
                .replace('\n', ' ')
                .replace('  ', ' ')
                .replace("'row[", "row['")
                .replace("]'", "']"))

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
    def build_queries(self, node_definitions: Dict, relationships: List) -> str:
        """Build Cypher queries with retry logic."""
        try:
            response = self.chain.invoke({
                "node_definitions": str(node_definitions),
                "relationships": str(relationships),
                "input": self.INPUT_EXAMPLE,
                "cypher": self.EXAMPLE_CYPHER
            })

            # Get response inside triple backticks
            if '```' in response:
                response = response.split('```')[1]


            # Sanitize response
            queries = self.sanitize_query(response)

            # Validate queries
            if not self.validate_cypher_query(queries):
                raise ValueError("Invalid Cypher query syntax")

            self.logger.info("Successfully generated Cypher queries")
            return queries

        except Exception as e:
            self.logger.error(f"Error building Cypher queries: {e}")
            raise

    def split_queries(self, queries: str) -> List[str]:
        """Split combined queries into individual statements."""
        return [q.strip() for q in queries.split(';') if q.strip()]

# Usage
builder = CypherQueryBuilder(llm=llm)
cypher_queries = builder.build_queries(node_definitions, relationships)
print("Cypher Queries:", cypher_queries)

In [ ]:
PROMPT_TEMPLATE = PromptTemplate(
    input_variables=["structure", "node_definitions", "relationships", "example"],
    template="""
        Consider the following Node Definitions:\n{node_definitions}\n\n
        Consider the following Relationships:\n{relationships}\n\n
        Generate Cypher queries to create nodes and relationships using the node definitions and relationships below. Remember to replace the placeholder values with actual data from the dataset.\n
        Include all the properties in the Node Definitions for each node as defined and create relationships.\n
        Return a single string with each query separated by a semicolon.\n
        Don't include any other text or quotation marks in the response.\n
        Please return only the string containing Cypher queries. Please do not report triple backticks to identify a code block.\n\n

        Example Input:\n{input}\n\n

        Example Output Cypher query:\n{cypher}
    """
)

In [ ]:
def __init__(self, llm: Any, logger: logging.Logger = None):
    self.llm = llm
    self.logger = logger or logging.getLogger(__name__)
    self.chain = self.PROMPT_TEMPLATE | self.llm

In [ ]:
def validate_cypher_query(self, query: str) -> bool:
    """Validate Cypher query syntax using LLM and regex patterns."""

    VALIDATION_PROMPT = PromptTemplate(
        input_variables=["query"],
        template="""
        Validate this Cypher query and return TRUE or FALSE:

        Query: {query}

        Rules to check:
        1. Valid CREATE statements
        2. Proper property formatting
        3. Valid relationship syntax
        4. No missing parentheses
        5. Valid property names
        6. Valid relationship types

        Return only TRUE if query is valid, FALSE if invalid.
        """
    )

    try:
        # Basic pattern validation
        basic_valid = all(re.search(pattern, query) for pattern in [
            r'CREATE \(',
            r'\{.*?\}',
            r'\)-\[:.*?\]->'
        ])

        if not basic_valid:
            return False

        # LLM validation
        validation_chain = VALIDATION_PROMPT | self.llm
        result = validation_chain.invoke({"query": query})

        # Parse result
        is_valid = "TRUE" in result.upper()

        if not is_valid:
            self.logger.warning(f"LLM validation failed for query: {query}")

        return is_valid

    except Exception as e:
        self.logger.error(f"Validation error: {e}")
        return False

In [ ]:
basic_valid = all(re.search(pattern, query) for pattern in [
    r'CREATE \(',
    r'\{.*?\}',
    r'\)-\[:.*?\]->'
])
if not basic_valid:
    return False

In [ ]:
validation_chain = VALIDATION_PROMPT | self.llm
result = validation_chain.invoke({"query": query})
is_valid = "TRUE" in result.upper()

In [ ]:
def sanitize_query(self, query: str) -> str:
    """Sanitize and format Cypher query."""
    return (query
            .strip()
            .replace('\n', ' ')
            .replace('  ', ' ')
            .replace("'row[", "row['")
            .replace("]'", "']"))

In [ ]:
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
    def build_queries(self, node_definitions: Dict, relationships: List) -> str:
        """Build Cypher queries with retry logic."""
        try:
            response = self.chain.invoke({
                "node_definitions": str(node_definitions),
                "relationships": str(relationships),
                "input": self.INPUT_EXAMPLE,
                "cypher": self.EXAMPLE_CYPHER
            })

            # Get response inside triple backticks
            if '```' in response:
                response = response.split('```')[1]


            # Sanitize response
            queries = self.sanitize_query(response)

            # Validate queries
            if not self.validate_cypher_query(queries):
                raise ValueError("Invalid Cypher query syntax")

            self.logger.info("Successfully generated Cypher queries")
            return queries

        except Exception as e:
            self.logger.error(f"Error building Cypher queries: {e}")
            raise

In [ ]:
if '```' in response:
    response = response.split('```')[1]

In [ ]:
def split_queries(self, queries: str) -> List[str]:
    """Split combined queries into individual statements."""
    return [q.strip() for q in queries.split(';') if q.strip()]

In [ ]:
# CREATE (n1:Movie {title: "Inception"}); CREATE (n2:Director {name: "Nolan"});

# ["CREATE (n1:Movie {title: 'Inception'})", "CREATE (n2:Director {name: 'Nolan'})"]

In [ ]:
builder = CypherQueryBuilder(llm=llm)
cypher_queries = builder.build_queries(node_definitions, relationships)
print("Cypher Queries:", cypher_queries)

In [ ]:
logs = ""
total_rows = len(df)

def sanitize_value(value):
    if isinstance(value, str):
        return value.replace('"', '')
    return str(value)

for index, row in tqdm(df.iterrows(),
                      total=total_rows,
                      desc="Loading data to Neo4j",
                      position=0,
                      leave=True):

    # Replace placeholders with actual values
    cypher_query = cypher_queries
    for column in df.columns:
        cypher_query = cypher_query.replace(
            f"row['{column}']",
            f'{sanitize_value(row[column])}'
        )

    try:
        # Execute query and update progress
        conn.execute_query(cypher_query)
    except Exception as e:
        logs += f"Error on row {index+1}: {str(e)}\n"

In [ ]:
def sanitize_value(value):
    if isinstance(value, str):
        return value.replace('"', '')
    return str(value)

In [ ]:
for index, row in tqdm(df.iterrows(),
                      total=total_rows,
                      desc="Loading data to Neo4j",
                      position=0,
                      leave=True):

    # Replace placeholders with actual values
    cypher_query = cypher_queries
    for column in df.columns:
        cypher_query = cypher_query.replace(
            f"row['{column}']",
            f'{sanitize_value(row[column])}'
        )

    try:
        # Execute query and update progress
        conn.execute_query(cypher_query)
    except Exception as e:
        logs += f"Error on row {index+1}: {str(e)}\n"

In [ ]:
llm_transformer = LLMGraphTransformer(
    llm=llm,
)

In [ ]:
df_sample = df.head(100)  # Reduce sample size for faster processing

In [ ]:
df_sample = movies.head(100)

documents = []
for _, row in tqdm(df_sample.iterrows(),
                   total=len(df_sample),
                   desc="Creating documents",
                   position=0,
                   leave=True):
    try:
        # Format text with proper line breaks
        text = ""

        for col in df.columns:
            text += f"{col}: {row[col]}\n"

        documents.append(Document(page_content=text))

    except KeyError as e:
        tqdm.write(f"Missing column: {e}")
    except Exception as e:
        tqdm.write(f"Error processing row: {e}")

In [ ]:
graph_documents = await llm_transformer.aconvert_to_graph_documents(documents)

In [ ]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

In [ ]:
graph = Neo4jGraph(url=uri, username=user, password=password, enhanced_schema=True)
graph.add_graph_documents(graph_documents)

In [ ]:
graph.refresh_schema()

In [ ]:
# llm = ChatGoogleGenerativeAI(
#     model="gemini-1.5-pro",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
#     api_key=api_key
# )

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Return every node as whole, do not return only the properties.

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,
    return_intermediate_steps=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT
)

In [ ]:
chain.run("Give me an overview of the movie titled David Copperfield.")